# Different ways to make a query on mongoDB with ***C#***
The idea is create the same query using different techniques and try understanding the benefits of each one.


## Setup 
___
***Reference***
* [mongo driver doc](https://docs.mongodb.com/drivers/csharp) 

### Dependencies

#### Mongo DB - DOCKER

In [ ]:
#!pwsh
docker run -d -p 27017-27017:27017-27017 mongo 

#### Nuget packages

In [ ]:
#r "nuget:MongoDB.Driver"

#### Client 

In [ ]:

using MongoDB.Driver;

var client = new MongoClient("mongodb://localhost:27017");
var database = client.GetDatabase("firstDatabase");

### Creating Models

In [ ]:
using MongoDB.Bson.Serialization.Attributes;

[BsonIgnoreExtraElements]
public class Collection{
    public string FirstProperty { get; set; }
    public int SecondProperty { get; set; }

}
[BsonIgnoreExtraElements]
public class FirstCollection : Collection{
    public IEnumerable<SecondCollection> SecondCollections { get; set; }
}

[BsonIgnoreExtraElements]
public class SecondCollection : Collection {
    public string Name { get; set; }
}

### Instanciate Collections

In [ ]:
var firstCollection = database.GetCollection<FirstCollection>("firstCollection");
var secondCollection = database.GetCollection<SecondCollection>("secondCollection");


### Seeding

#### first Collection

In [ ]:

var firstCollections = new List<FirstCollection>{
    new FirstCollection {FirstProperty = "First", SecondProperty = 1 },
    new FirstCollection {FirstProperty = "Second", SecondProperty = 2 },
    new FirstCollection {FirstProperty = "Third", SecondProperty = 3 },
    new FirstCollection {FirstProperty = "Fourth", SecondProperty = 4 }
};

firstCollection.InsertMany(firstCollections);

#### second Collection

In [ ]:

var secondCollections = new List<SecondCollection>{
    new SecondCollection {FirstProperty = "First", SecondProperty = 1, Name = "I" },
    new SecondCollection {FirstProperty = "Second", SecondProperty = 2 , Name = "II"},
    new SecondCollection {FirstProperty = "Third", SecondProperty = 3 , Name = "III"},
    new SecondCollection {FirstProperty = "Fourth", SecondProperty = 4 , Name = "IV"}
};

secondCollection.InsertMany(secondCollections);

### Until methods

In [ ]:
 public void PrintResult<TResult>(IEnumerable<TResult> results, Func<TResult,string> output){
        Console.WriteLine($"Result Count: {results.Count()}");
        results.ToList().ForEach(res => Console.WriteLine(output(res)));
   
    public void PrintResult(IEnumerable<FirstCollection> results) => PrintResult(results, result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondPropert    ;
    public void PrintResult(IEnumerable<string> results) => PrintResult(results, result => $"Name: {result}");


## The queries
We'll use two queries as base, first one Simple query with just a composed filter and the last one is Aggregation query with a simple filter and an aggregation with two collections. That queries will used for exemplify complexity and readability each expression construction.  

explain each query: 
1. The **Simple query**: Filter ***FirstCollection*** by property `FirstProperty` not equals ***'First'*** and the property `SecondProperty` in 1,2 an

~~~csharp
    db.firstCollection.find({
       FirstProperty: {
            $ne: "First"
        },
        SecondProperty:{
            $in:[1,2,4]
        }

    })
~~~
2. The **Aggregate query**: Aggregate the  ***FirstCollection*** with ***SecondCollection*** by `FirstProperty` and filter ***FirstCollection*** by property `FirstProperty` not equals ***'First'***, selecting only ***SecondColle

~~~csharp
    db.firstCollection.aggregate([
        {
            $match: {
                FirstProperty: {
                    $ne: "First"
                }
   
                   }, {
            $lookup: {
                from: 'secondCollection',
                localField: 'SecondProperty',
                foreignField: 'SecondProperty',
                as: 'seconds'

        }
        }
    ])
~~~

### Using ***Filters*** Builder 
***Reference:***
* [Mongo DB Blog](https://www.mongodb.com/blog/post/quick-start-c-and-mongodb--read-operations)

#### The Simple Query 


In [ ]:
var numbers = new List<int>{1,4,2};

var collectionInFilter = Builders<FirstCollection>.Filter.In(collect => collect.SecondProperty,numbers);
var collectionEqFilter = Builders<FirstCollection>.Filter.Eq(collect => collect.FirstProperty,"First");
var collectionNotInFilter = Builders<FirstCollection>.Filter.Not(collectionEqFilter);
var collectionAndFilter = Builders<FirstCollection>.Filter.And(collectionInFilter,collectionNotInFilter);

var findFilterResult = firstCollection.FindSync(collectionAndFilter).ToList();

PrintResult(findFilterResult);

#### The Aggregate Query 

In [ ]:
var numbers = new List<int>{1,4,2};

var collectionEqFilter = Builders<FirstCollection>.Filter.Eq(collect => collect.FirstProperty,"First");
var collectionNotInFilter = Builders<FirstCollection>.Filter.Not(collectionEqFilter);


var findFilterResult = firstCollection.Aggregate()
    .Match(collectionNotInFilter)
    .Lookup(
        foreignCollection: secondCollection,
        first => first.SecondProperty,
        second => second.SecondProperty,
        @as : (FirstCollection first) => first.SecondCollections)
    .Project(x => new {Names = x.SecondCollections.Select(second => second.Name)})
    .ToList()
    .SelectMany(project => project.Names);

PrintResult(findFilterResult);

### Using Method based LINQ 

#### Explanning 

Using a **Fluent interface**, **Method based LINQ** has closest expression **MongoDB** regular queries than **Query based LINQ** principally by use of Lambda expressions (**Higher order functions**) to filter's logic.The combination between Lambda expressions and Fluent interface result is code verbosity and code noise but easy to understanding for C# developers.
MongoDB offers others methods to query, so is possible to using Lambda expressions to simple queries without use LINQ. Look below an example of likeness between **Method based LINQ**, **MongoDB** regular queries and **MongoBD Driver** methods:  
* **MongoDB:**
~~~javascript
    db.collection.find({
        propertyA:1,
        propertyB:{$ne:"VALUE"}
    })
~~~
* **LINQ:**
~~~csharp
    collection.AsQueryable()
        .Where(collection => 
            collection.PropertyA == 1 
            && collection.PropertyB != "VALUE"
            )
        .ToList();
~~~
* **MongoDB Driver:**
~~~csharp
    collection
        .FindSync(collection => 
            collection.PropertyA == 1 
            && collection.PropertyB != "VALUE"
            )
        .ToList();
~~~  

**Contextualizing:**
* **Fluent interface**: Is a chain of methods that return a same Type.
* **Higher order function**: Basically is a use of a Function as parameter of another function.

***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/)
* [Fluent interface](https://en.wikipedia.org/wiki/Fluent_interface#:~:text=In%20software%20engineering%2C%20a%20fluent,Eric%20Evans%20and%20Martin%20Fowler.)
* [Higher order function](https://en.wikipedia.org/wiki/Higher-order_function)

#### The Simple Query 


In [ ]:
var numbers = new List<int>{1,4,2};

var fluentResult = firstCollection.AsQueryable()
    .Where(collect => 
        !collect.FirstProperty.Equals("First")
        && numbers.Contains(collect.SecondProperty))
    .ToList();

PrintResult(fluentResult);


When has a simple query all power and expressive of **Method based LINQ** is shown with few lines. But MongoDB driver already have a method to simple queries `FindSync` and `FindAsync`, so **Method based LINQ** is necessary in this situation? 🤔

#### The Aggregate Query 

In [ ]:
var numbers = new List<int>{1,4,2};

var fluentResult = firstCollection.AsQueryable()
    .Where(collect => !collect.FirstProperty.Equals("First"))
    .Join(secondCollection.AsQueryable(),
        first => first.SecondProperty,
        second => second.SecondProperty,
        (_,second) => second)
    .Select(second => second.Name)
    .ToList();

PrintResult(fluentResult);


### Using (Query based) ***Mongo driver*** LINQ 


#### Explaining 

Expressive, With a familiar syntax and noiseless than **C#**, Query based syntax LINQ use SQL syntax like to create a better code readability.  
The query construction and syntax have little differences between LINQ and SQL, to create an alias for example is used the statement `from` following the alias name and the statement `in` is used as collection's pointer, not the statement `from` as SQL language. Another good example is the order of query expression construction, in SQL the properties selection come first than filter, but in LINQ first is created an alias, obligatory to access an object in collection, after the filter is created and finally the object selection.  

Simple Query example:
* **SQL:**
~~~sql
    SELECT * 
        FROM COLLECTION AS COLL 
        WHERE COLL.PROPERTY = 1
~~~
* **LINQ:**
~~~csharp
    var result = from coll in collection
                where coll.Property == 1
                select coll
~~~

**Contextualizing:**
* **DSL**: **D**omain **S**pecific **L**anguage is a higher level language optimized to solve specific problems, can used as standalone language or integrated in another programming languages.  
* **Declarative Programming**: A programming paradigm or programming style, describe the problem solution without implementations details like **SQL** language.

**References:**
* [Declarative programming](https://getpocket.com/redirect?url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FDeclarative_programming)
* [Domain-Specific Languages](https://www.jetbrains.com/mps/concepts/domain-specific-languages/)
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/) 
* [LINQ as DSL](https://app.getpocket.com/read/652237004)
* [Query Syntax and Method Syntax in LINQ](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/linq/query-syntax-and-method-syntax-in-linq)
___

#### The Simple Query 

In [ ]:
var numbers = new List<int>{1,4,2};

var dlsResult = from collect in firstCollection.AsQueryable()
                where collect.FirstProperty != "First"
                     && numbers.Contains(collect.SecondProperty)
                select collect;

PrintResult(dlsResult.ToList());


Here the expression is more verbosity than **Method based LINQ** because the alias obligatory and code's readability was impaired by C# boolean operators `&&` and `!`, removing that little issues is easy to understand the expression.  
That make me think, is good a choice use **Query based LINQ** to simple queries or is better use **Method based LINQ** or **MongoDB Driver** methods? 🤔

#### The Aggregate Query 

In [ ]:
var numbers = new List<int>{1,4,2};

var dlsResult = from second in secondCollection.AsQueryable()
                join first in firstCollection.AsQueryable()
                    on second.SecondProperty equals first.SecondProperty
                where first.FirstProperty != "First"
                orderby first.SecondProperty
                select second.Name;

PrintResult(dlsResult.ToList());

The big deal of **Query based LINQ**, an aggregation query.  
Using a **SQL** like code style give a better readability principally because the use of `join` as aggregate statement followed of `on` statement to relate properties. Different than **Method based LINQ**, a **Query based LINQ** expression the relation statement and filter statement are made together at end make it better organized, in other words the expression can made at once.  

Look below the same query with **SQL**:  
~~~sql
    SELECT SECOND.NAME 
    FROM FIRSTCOLLECTION AS FIRST
    INNER JOIN SECONDCOLLECTION AS SECOND
        ON FIRST.SECONDPROPERTY = SECOND.SECONDPROPERTY
    WHERE FIRST.FIRSTPROPERTY <> 'First'
    ORDER BY FIRST.SECONDPROPERTY
~~~  